In [52]:
import pandas as pd
import os
import requests
import json

In [93]:
# get authentification header without displaying bearer token 

bearer_token = os.environ.get('BEARER_TOKEN')
def create_headers(bearer_token):
    return {"Authorization": "Bearer {}".format(bearer_token)}   

In [94]:
# parse JSON payload into data frame
      
def make_df(response):
    return pd.DataFrame(response["data"])

In [95]:
def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [96]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))

In [110]:
def set_rules(headers, delete, bearer_token):
    # rules for retrieving tweets
    
    sample_rules = [
        {"value": "(bitcoin OR bitcoins OR btc) (-giveaway -airdrop) lang:en", "tag": "btc",
        "expansions" : "author_id,geo.place_id",
        "user.fields": "public_metrics,verified,username,created_at",
        "place.fields" : "country"
        }
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [111]:
def get_stream(headers, set, bearer_token):
    params = {
        # feature selection
        "tweet.fields": "author_id,in_reply_to_user_id,lang,"
                        + "conversation_id,created_at,geo,"
                        + "public_metrics,entities,referenced_tweets",
    }
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream",
        headers=headers, params=params, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
        
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)            
            print(json.dumps(json_response, indent=4, sort_keys=True))
    
    return response.json()

In [112]:
def make_df(response):
    return pd.DataFrame(response["data"])

In [113]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    response = get_stream(headers, set, bearer_token)
    df = make_df(response)
    print(df)


if __name__ == "__main__":
    main()

{"meta": {"sent": "2021-06-16T21:17:41.714Z"}}
{"data": [{"value": "(#bitcoin OR #btc OR #bitcoins OR #bitcoinnewsOR #bitcointrading) lang:en sample:1", "tag": "btc", "id": "1405273400738824196"}], "meta": {"sent": "2021-06-16T21:17:43.107Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))